In [1]:
# !wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
# !unzip uncased_L-12_H-768_A-12.zip

--2019-07-05 04:30:40--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 2404:6800:4003:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  56.7MB/s    in 7.7s    

2019-07-05 04:30:48 (50.4 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [37]:
!ls -lh uncased_L-12_H-768_A-12

total 422M
-rw-r--r-- 1 husein husein  313 Oct 18  2018 bert_config.json
-rw-r--r-- 1 husein husein 421M Oct 18  2018 bert_model.ckpt.data-00000-of-00001
-rw-r--r-- 1 husein husein 8.4K Oct 18  2018 bert_model.ckpt.index
-rw-r--r-- 1 husein husein 884K Oct 18  2018 bert_model.ckpt.meta
-rw-r--r-- 1 husein husein 227K Oct 18  2018 vocab.txt


In [5]:
import json

with open('uncased_L-12_H-768_A-12/bert_config.json') as fopen:
    config = json.load(fopen)
config

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [20]:
config['num_hidden_layers'] = 6
with open('6_bert_config.json', 'w') as fopen:
    json.dump(config, fopen)

In [6]:
from utils import *
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import time
import random
import os

/home/husein/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [21]:
BERT_VOCAB = 'uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = '6_bert_config.json'

In [9]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [10]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

In [11]:
MAX_SEQ_LENGTH = 100

In [12]:
from tqdm import tqdm

input_ids, input_masks, segment_ids = [], [], []

for text in tqdm(trainset.data):
    tokens_a = tokenizer.tokenize(text)
    if len(tokens_a) > MAX_SEQ_LENGTH - 2:
        tokens_a = tokens_a[:(MAX_SEQ_LENGTH - 2)]
    tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
    segment_id = [0] * len(tokens)
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_id)
    padding = [0] * (MAX_SEQ_LENGTH - len(input_id))
    input_id += padding
    input_mask += padding
    segment_id += padding
    
    input_ids.append(input_id)
    input_masks.append(input_mask)
    segment_ids.append(segment_id)

100%|██████████| 10662/10662 [00:03<00:00, 3454.79it/s]


In [25]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [14]:
tokenizer.tokenize(trainset.data[0])

['ra',
 '##ra',
 'av',
 '##is',
 'intelligent',
 'romantic',
 'comedy',
 'actual',
 'ideas',
 'mind']

In [15]:
epoch = 5
batch_size = 3
warmup_proportion = 0.1
num_train_steps = int(len(input_ids) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [26]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=True,
            input_ids=self.X,
            input_mask=self.input_masks,
            token_type_ids=self.segment_ids,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_pooled_output()
        self.logits = tf.layers.dense(output_layer, dimension_output)
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [27]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())

In [28]:
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(30522, 768) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/bia

In [29]:
def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue
        assignment_map[name] = name_to_variable[name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [30]:
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                BERT_INIT_CHKPNT)
initialized_variable_names

{'bert/embeddings/LayerNorm/beta': 1,
 'bert/embeddings/LayerNorm/beta:0': 1,
 'bert/embeddings/LayerNorm/gamma': 1,
 'bert/embeddings/LayerNorm/gamma:0': 1,
 'bert/embeddings/position_embeddings': 1,
 'bert/embeddings/position_embeddings:0': 1,
 'bert/embeddings/token_type_embeddings': 1,
 'bert/embeddings/token_type_embeddings:0': 1,
 'bert/embeddings/word_embeddings': 1,
 'bert/embeddings/word_embeddings:0': 1,
 'bert/encoder/layer_0/attention/output/LayerNorm/beta': 1,
 'bert/encoder/layer_0/attention/output/LayerNorm/beta:0': 1,
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma': 1,
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma:0': 1,
 'bert/encoder/layer_0/attention/output/dense/bias': 1,
 'bert/encoder/layer_0/attention/output/dense/bias:0': 1,
 'bert/encoder/layer_0/attention/output/dense/kernel': 1,
 'bert/encoder/layer_0/attention/output/dense/kernel:0': 1,
 'bert/encoder/layer_0/attention/self/key/bias': 1,
 'bert/encoder/layer_0/attention/self/key/bias:0': 1

In [32]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, BERT_INIT_CHKPNT)

INFO:tensorflow:Restoring parameters from uncased_L-12_H-768_A-12/bert_model.ckpt


In [33]:
train_input_ids, test_input_ids, train_input_masks, test_input_masks, train_segment_ids, test_segment_ids, train_Y, test_Y = train_test_split(
    input_ids, input_masks, segment_ids, trainset.target, test_size = 0.2
)

In [34]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_masks = train_input_masks[i: index]
        batch_segment = train_segment_ids[i: index]
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_masks = test_input_masks[i: index]
        batch_segment = test_segment_ids[i: index]
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_input_ids) / batch_size
    train_acc /= len(train_input_ids) / batch_size
    test_loss /= len(test_input_ids) / batch_size
    test_acc /= len(test_input_ids) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 1/2843 [00:00<08:03,  5.88it/s, accuracy=1, cost=0.0483]

epoch: 0, pass acc: 0.000000, current acc: 0.765588
time taken: 494.7259032726288
epoch: 0, training loss: 0.679368, training acc: 0.699144, valid loss: 0.647409, valid acc: 0.765588



train minibatch loop:   0%|          | 1/2843 [00:00<07:56,  5.97it/s, accuracy=1, cost=0.0641]

epoch: 1, pass acc: 0.765588, current acc: 0.776371
time taken: 494.1690831184387
epoch: 1, training loss: 0.536140, training acc: 0.850745, valid loss: 0.849323, valid acc: 0.776371



train minibatch loop:   0%|          | 1/2843 [00:00<07:59,  5.93it/s, accuracy=1, cost=0.000441]

time taken: 494.4359619617462
epoch: 2, training loss: 0.324216, training acc: 0.929417, valid loss: 1.227605, valid acc: 0.773558



train minibatch loop:   0%|          | 1/2843 [00:00<07:59,  5.93it/s, accuracy=1, cost=2.95e-5]

time taken: 494.08778500556946
epoch: 3, training loss: 0.158032, training acc: 0.968109, valid loss: 1.683650, valid acc: 0.748242



test minibatch loop: 100%|██████████| 711/711 [00:22<00:00, 31.02it/s, accuracy=0.667, cost=2.24] 

time taken: 493.2808885574341
epoch: 4, training loss: 0.088819, training acc: 0.982765, valid loss: 1.804691, valid acc: 0.773090

break epoch:5



In [35]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'bert/model.ckpt')

'bert/model.ckpt'

In [36]:
!ls -lh bert

total 258M
-rw-rw-r-- 1 husein husein   77 Jul  5 06:47 checkpoint
-rw-rw-r-- 1 husein husein 256M Jul  5 06:47 model.ckpt.data-00000-of-00001
-rw-rw-r-- 1 husein husein 4.3K Jul  5 06:47 model.ckpt.index
-rw-rw-r-- 1 husein husein 2.0M Jul  5 06:47 model.ckpt.meta


In [38]:
(421 - 256) / 421

0.3919239904988123

## We save 40% of space, accuracy drop around 3-5%